# Assignmnet 2 (100 points)

**Name: Kotomi Fukushima** <br>
**Email: kof6267@thi.de** <br>
**Group:** A <br>
**Hours spend *(optional)* :** <br>

### SMS Spam Detection *(60 points)*

<p>You are hired as an AI expert in the development department of a telecommunications company. The first thing on your orientation plan is a small project that your boss has assigned you for the following given situation. Your supervisor has given away his private cell phone number on too many websites and is now complaining about daily spam SMS. Therefore, it is your job to write a spam detector in Python. </p>

<p>In doing so, you need to use a Naive Bayes classifier that can handle both bag-of-words (BoW) and tf-idf features as input. For the evaluation of your spam detector, an SMS collection is available as a dataset - this has yet to be suitably split into train and test data. To keep the costs as low as possible and to avoid problems with copyrights, your boss insists on a new development with Python.</p>

<p>Include a short description of the data preprocessing steps, method, experiment design, hyper-parameters, and evaluation metric. Also, document your findings, drawbacks, and potential improvements.</p>

<p>Note: You need to implement the bag-of-words (BoW) and tf-idf feature extractor from scratch. You can use existing python libraries for other tasks.</p>

**Dataset and Resources**

* SMS Spam Collection Dataset: https://archive.ics.uci.edu/dataset/228/sms+spam+collection

In [33]:
import math
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report



def load_data(path):
    labels = []
    messages = []

    with open(path, "r", encoding = "utf-8") as file:
        for line in file:
            label, message = line.strip().split('\t')  
            labels.append(1 if label == "spam" else 0)  
            messages.append(message)  
    
    return labels, messages
            


def tokenize(text): #remove punctuations and symbols
    text = text.lower()
    clean_text = "" #initialize clean text
    for char in text:
        if char.isalnum() or char.isspace():
            clean_text += char
        else:
            clean_text += ""
    clean_text = clean_text.split()
    return clean_text



def bag_of_words(documents, vocab=None):
    tokenized_docs = []
    vectors = []
    vocab_index = {}


    # make a list of tokenized documents 
    for doc in documents:
        tokens = tokenize(doc)
        tokenized_docs.append(tokens)
            
    if vocab is None:
        vocab = set()
        # make a set of all unique word
        for tokens in tokenized_docs:
            vocab.update(tokens) 
        vocab = list(vocab)

    for i, word in enumerate(vocab):
            vocab_index[word] = i

    for tokens in tokenized_docs:
        vector = [0] * len(vocab) # create a vector of len(vocab) zeros
        for word in tokens:
            if word in vocab_index:
                vector[vocab_index[word]] += 1
        vectors.append(vector)

    return vectors, vocab
    

def tf_idf(documents, vocab=None, vocab_index=None, idf_values=None):
    tokenized_docs = []
    vocab_index = {}
    
    # make a list of tokenized documents 
    for doc in documents:
        tokens = tokenize(doc)
        tokenized_docs.append(tokens)

    if vocab is None:
        vocab = set()
        for tokens in tokenized_docs:
            vocab.update(tokens)
        vocab = list(vocab)

        for i, word in enumerate(vocab):
            vocab_index[word] = i
    
        doc_count = [0] * len(vocab)
    
        for tokens in tokenized_docs:
            unique_word = set(tokens)
            for token in unique_word:
                index = vocab_index[token]
                doc_count[index] += 1

        N = len(documents)

        idf = [0.0] * len(vocab)
        for i, df in enumerate(doc_count):
            idf[i] = math.log((N + 1) / (df + 1)) + 1

    tf_idf_vectors = []
    for token in tokenized_docs:
        tf = [0] * len(vocab)
        for token in tokens:
            if token in vocab_index:
                index = vocab_index[token]
                tf[index] += 1

        vector = [0.0] * len(vocab)
        for i in range(len(vocab)):
            if tf[i] > 0:
                vector[i] = (tf[i] / len(tokens)) * idf[i]
        tf_idf_vectors.append(vector)

    return tf_idf_vectors, vocab, vocab_index, idf_values



# split the data into train and test data
labels, messages = load_data("sms_spam_collection\SMSSpamCollection")
mes_train, mes_test, lab_train, lab_test = train_test_split(messages, labels, test_size=0.2, random_state=0)

# train naive bayes classifier
model = MultinomialNB()

# # use BOW
# mes_train_bow, vocab = bag_of_words(mes_train)
# mes_test_bow, _ = bag_of_words(mes_test, vocab=vocab)

# model.fit(mes_train_bow, lab_train)
# predictions = model.predict(mes_test_bow) 
# print("Accuracy for BoW:", accuracy_score(lab_test, predictions))
# print("Classification Report for BoW:\n", classification_report(lab_test, predictions))



# use TF-IDF
mes_train_tfidf, vocab, vocab_index, idf_values = tf_idf(mes_train)
mes_test_tfidf, _, _, _ = tf_idf(mes_test, vocab, vocab_index, idf_values)

model.fit(mes_train_tfidf, lab_train) 
predictions = model.predict(mes_test_tfidf) 
print("Accuracy for TF-IDF:", accuracy_score(lab_test, predictions))
print("Classification Report for TF-IDF:\n", classification_report(lab_test, predictions, zero_division = 0))


            




Accuracy for BoW: 0.9820627802690582
Classification Report for BoW:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       950
           1       0.98      0.90      0.94       165

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



 ### Search Engine *(40 points)*

Your boss is impressed with your spam detector and assigns you a new task. As part of improving internal tools, the company wants a search engine that can search through SMS messages and rank them by relevance. Implement the PageRank algorithm from scratch to score each SMS message based on its importance in the document graph.

*   Compute TF-IDF vectors for all SMS messages (you can leverage previous implementation)
*   Construct a document graph, where each node represents an SMS message and edges are the links between nodes.
*  Implement the PageRank algorithm from scratch to assign an importance score to each SMS message based on its position in the document graph.

#### Hint : You can use the previous dataset or any dataset from your choice.




## You might need the follwoing formulas for your implementation

---

### 1) Cosine Similarity Between Two Document Vectors

Cosine similarity measures how similar two vectors are based on the angle between them:

$$
\text{cosine\_sim}(A, B) = \frac{A \cdot B}{\|A\| \cdot \|B\|}
$$

- \( A \cdot B \): Dot product of vectors \( A \) and \( B \)  
- \( \|A\| \): Euclidean norm (magnitude) of vector \( A \)  
- \( \|B\| \): Euclidean norm of vector \( B \)

**Use case**: Comparing TF-IDF vectors to measure similarity between two messages.

---

### 2) PageRank of a Node \( i \)

PageRank estimates the importance of a document based on its connections in a graph:

$$
PR(i) = \frac{1 - d}{N} + d \sum_{j \in M(i)} \frac{PR(j)}{L(j)}
$$

Where:
- \( PR(i) \): PageRank score of node \( i \)  
- \( d \): Damping factor (typically 0.85)  
- \( N \): Total number of nodes (documents) in the graph  
- \( M(i) \): Set of nodes that link to node \( i \)  
- \( L(j) \): Number of outbound links from node \( j \)  

**Interpretation**:  
- A document is important if **important documents link to it**.  
- The score is split among a node’s outbound links.  
- The **teleportation term** $\text(\frac{1 - d}{N})$ accounts for random jumps, ensuring stability and fairness.

---



In [32]:
import numpy as np



def cosine_similarity_calculation(vec_x, vac_y):
    dot_product = np.dot(vec_x, vac_y)
    norm_x = np.linalg.norm(vec_x)
    norm_y = np.linalg.norm(vac_y)
    return dot_product / (norm_x * norm_y) if norm_x * norm_y != 0 else 0.0

def document_graph(tf_idf_vectors, threshold=0.1):
    N = len(tf_idf_vectors)
    graph = {}
    for i in range(N):
        graph[i] = []

    for i in range(N):
        for j in range(i + 1, N):
            sim = cosine_similarity_calculation(tf_idf_vectors[i], tf_idf_vectors[j])
            if sim > threshold:
                graph[i].append(j)
                graph[j].append(i)
    
    return graph

def pagerank(graph, damping_factor=0.85, max_iter=100, tol=1e-6):
    N = len(graph)
    pr_scores = {i: 1 / N for i in range(N)}  # Initialize with equal scores
    teleport = (1 - damping_factor) / N
    
    for _ in range(max_iter):
        new_pr_scores = {}
        
        for node in range(N):
            rank_sum = 0
            for neighbor in graph[node]:
                rank_sum += pr_scores[neighbor] / len(graph[neighbor])
            new_pr_scores[node] = teleport + damping_factor * rank_sum
        
        diff = sum(abs(new_pr_scores[node] - pr_scores[node]) for node in range(N))
        if diff < tol:
            break
        
        pr_scores = new_pr_scores

    return pr_scores

def rank_sms_messages(messages):
    tf_idf_vectors, vocab, vocab_index, idf_values = tf_idf(messages)
    graph = create_document_graph(tf_idf_vectors)
    pr_scores = pagerank(graph)
    ranked_sms = sorted(pr_scores.items(), key=lambda x: x[1], reverse=True)
    
    return ranked_sms

messages = messages[:100]

#Compute TF-IDF vectors for all SMS messages
tf_idf_vectors, vocab, vocab_index, idf_values = tf_idf(messages)

graph = document_graph(tf_idf_vectors, threshold=0.1)

pr_scores = pagerank(graph)

ranked_sms = sorted(pr_scores.items(), key=lambda x: x[1], reverse=True)

for idx, score in ranked_sms:  
    print(f"Message {idx}: {messages_subset[idx][:60]}... - PageRank: {score:.4f}")




#reference https://www.datacamp.com/tutorial/python-bag-of-words-model  chatGPT

Message 0: Go until jurong point, crazy.. Available only in bugis n gre... - PageRank: 0.0100
Message 1: Ok lar... Joking wif u oni...... - PageRank: 0.0100
Message 2: Free entry in 2 a wkly comp to win FA Cup final tkts 21st Ma... - PageRank: 0.0100
Message 3: U dun say so early hor... U c already then say...... - PageRank: 0.0100
Message 4: Nah I don't think he goes to usf, he lives around here thoug... - PageRank: 0.0100
Message 5: FreeMsg Hey there darling it's been 3 week's now and no word... - PageRank: 0.0100
Message 6: Even my brother is not like to speak with me. They treat me ... - PageRank: 0.0100
Message 7: As per your request 'Melle Melle (Oru Minnaminunginte Nurung... - PageRank: 0.0100
Message 8: WINNER!! As a valued network customer you have been selected... - PageRank: 0.0100
Message 9: Had your mobile 11 months or more? U R entitled to Update to... - PageRank: 0.0100
Message 10: I'm gonna be home soon and i don't want to talk about this s... - PageRank: 0.0100
Message

### Additional Experiments *(5 additional points - <span style="color: red;">Optional</span>)*